In [25]:
# config.ipynb cell
import os
import sys
import supabase

# Dynamically add the parent directory to the Python path
current_dir = os.path.dirname(os.path.abspath(__name__))
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import uuid
import random
import json
from faker import Faker
from datetime import datetime
from supabase import create_client, Client
from src.api.config import settings

# Supabase credentials
SUPABASE_URL = settings.SUPABASE_URL
SUPABASE_KEY = settings.SUPABASE_ANON_KEY
client = create_client(SUPABASE_URL, SUPABASE_KEY)


🔍 2. Pick a Random Test User

In [36]:
import random

# Fetch a few users to test
users = client.table("user_profiles").select("user_id").limit(10).execute().data
test_user_id = random.choice(users)["user_id"]
print("🔍 Testing for user_id:", test_user_id)


🔍 Testing for user_id: 969d3ddd-59b7-4e5f-81a6-cfd5e72f96d7


🧠 3. Matchmaking Logic

In [38]:
from datetime import datetime

def match_top(user_id: str, top_k: int = 5):
    user = client.table("user_profiles").select("*").eq("user_id", user_id).single().execute().data
    if not user:
        return {"error": "User not found"}

    budget_min = user["budget_min"]
    budget_max = user["budget_max"]
    location = user["location_preference"]
    lifestyle_tags = set(user.get("lifestyle_tags") or [])

    roommates = client.table("user_profiles") \
        .select("*") \
        .neq("user_id", user_id) \
        .eq("location_preference", location) \
        .gte("budget_max", budget_min) \
        .lte("budget_min", budget_max) \
        .execute().data

    properties = client.table("properties") \
        .select("*") \
        .eq("location", location) \
        .gte("price", budget_min) \
        .lte("price", budget_max) \
        .lte("available_from", datetime.utcnow().isoformat()) \
        .execute().data

    def roommate_score(rm):
        rm_tags = set(rm.get("lifestyle_tags") or [])
        tag_score = len(lifestyle_tags & rm_tags) / len(lifestyle_tags | rm_tags) if lifestyle_tags and rm_tags else 0
        rm_budget_avg = (rm["budget_min"] + rm["budget_max"]) / 2
        user_budget_avg = (budget_min + budget_max) / 2
        budget_score = 1 - abs(user_budget_avg - rm_budget_avg) / max(user_budget_avg, rm_budget_avg)
        return round(0.5 * budget_score + 0.5 * tag_score, 3)

    def property_score(prop):
        prop_amenities = set(prop.get("amenities") or [])
        amenity_score = len(lifestyle_tags & prop_amenities) / len(lifestyle_tags | prop_amenities) if lifestyle_tags and prop_amenities else 0
        price_score = 1 - abs(((budget_min + budget_max) / 2) - prop["price"]) / budget_max
        return round(0.7 * price_score + 0.3 * amenity_score, 3)

    top_roommates = sorted(roommates, key=roommate_score, reverse=True)[:top_k]
    top_properties = sorted(properties, key=property_score, reverse=True)[:top_k]

    return {
        "roommate_matches": [
            {"user_id": rm["user_id"], "score": roommate_score(rm)} for rm in top_roommates
        ],
        "property_matches": [
            {"property_id": prop["id"], "score": property_score(prop)} for prop in top_properties
        ]
    }
# Example usage

In [51]:
k = 5
results = match_top(user_id=test_user_id, top_k=k)


In [52]:
results

{'roommate_matches': [{'user_id': '0f5576b4-bd2a-41de-aaac-eab6e28073b9',
   'score': 0.987},
  {'user_id': 'd86ee1c8-2edb-4696-a6c4-6488f651a699', 'score': 0.96},
  {'user_id': 'e44020ec-a478-4381-81a6-419bfc4bd7f8', 'score': 0.823},
  {'user_id': '9de7fac9-847f-41d3-92e3-6be8e5dd544b', 'score': 0.811},
  {'user_id': '1eddebe4-e8b8-47ca-b0e0-2da43147afa4', 'score': 0.795}],
 'property_matches': [{'property_id': 294, 'score': 0.755},
  {'property_id': 227, 'score': 0.74},
  {'property_id': 761, 'score': 0.732},
  {'property_id': 586, 'score': 0.709},
  {'property_id': 593, 'score': 0.708}]}

In [53]:
print(f"🔍 Top K roommates and properties for UserID: {test_user_id}\n")
# print for k - 5
print(f"For K = {k}, found {len(results['roommate_matches'])} roommates and {len(results['property_matches'])} properties.\n")

print("🏠 Top Property Matches:")
for idx, match in enumerate(results["property_matches"], start=1):
    print(f"{idx}. Property ID: {match['property_id']}, Match Score: {match['score']}")

print("\n👥 Top Roommate Matches:")
for idx, match in enumerate(results["roommate_matches"], start=1):
    print(f"{idx}. User ID: {match['user_id']}, Match Score: {match['score']}")

🔍 Top K roommates and properties for UserID: 969d3ddd-59b7-4e5f-81a6-cfd5e72f96d7

For K = 5, found 5 roommates and 5 properties.

🏠 Top Property Matches:
1. Property ID: 294, Match Score: 0.755
2. Property ID: 227, Match Score: 0.74
3. Property ID: 761, Match Score: 0.732
4. Property ID: 586, Match Score: 0.709
5. Property ID: 593, Match Score: 0.708

👥 Top Roommate Matches:
1. User ID: 0f5576b4-bd2a-41de-aaac-eab6e28073b9, Match Score: 0.987
2. User ID: d86ee1c8-2edb-4696-a6c4-6488f651a699, Match Score: 0.96
3. User ID: e44020ec-a478-4381-81a6-419bfc4bd7f8, Match Score: 0.823
4. User ID: 9de7fac9-847f-41d3-92e3-6be8e5dd544b, Match Score: 0.811
5. User ID: 1eddebe4-e8b8-47ca-b0e0-2da43147afa4, Match Score: 0.795
